In [14]:
#python train.py --model_name ResNet101 --labels vine grass --start_date 2021-03-01 --end_date 2021-08-01 --experiment_name experiment_test 

#import requests
#from keras.applications import ResNet50
#from keras.applications.resnet import decode_predictions, preprocess_input
#from keras.preprocessing.image import load_img, img_to_array
#import numpy as np
import os
import sys
from dotenv import load_dotenv
#import pandas as pd
#from sklearn.model_selection import train_test_split
#from datetime import datetime
#from helpers import *
from build_dataset import *
from build_model import *
import argparse
from datetime import datetime
import mlflow

In [15]:
# We load the environment variables from the secret.env file
load_dotenv()

# We access environment variables using os.getenv()
api_key = os.getenv("API_KEY")
api_url = os.getenv("API_URL")
mlflow_tracking_uri = os.getenv("MLFLOW_TRACKING_URI")

model_name = "ResNet50"
# We compile the model
model = compile_new_model(model_name)

Model 'ResNet50' found in tf.keras.applications.
Base_model 'ResNet50' loaded successfully.
New ResNet50 compiled successfully and is ready to be trained!


In [16]:
image_dir = "media"
labels = ["vine", "grass"]
start_date = "2021-03-01"
end_date = "2021-05-01"
# We make the train and validation datasets
image_urls = get_image_urls_with_multiple_labels(labels, start_date, end_date, api_key, api_url)
# Download images and create a sample map
df_sample_map = create_sample_map(image_urls)
df_sample_map = download_images(df_sample_map, image_dir)

Number of urls collected for vine: 1
Number of urls collected for grass: 28
Dataframe created successfully with shape : (29, 4)


In [21]:
train_dataset, val_dataset = create_train_val_datasets(df_sample_map,
                              image_dir = 'media',
                              model_name = model_name,
                              )

Preprocess_input function for 'ResNet50' loaded successfully.


In [22]:
# Convert DataFrame to JSON
json_data = df_sample_map.to_json(orient="records")

experiment_name = "experiment_test"

# We check if an experiment name is provided, if so we log the data to MLflow
if experiment_name is not None:
    
    # We set the tracking URI
    mlflow.set_tracking_uri("sqlite:///mlflow.db")
    
    # We set the experiment name
    mlflow.set_experiment(experiment_name)
    
    # Save JSON data to a file
    with open("dataset_json.json", "w") as f:
        f.write(json_data)
    
    # We set the run name
    run_name = f"{model_name}_"+datetime.now().strftime("%Y%m%d-%H%M%S")
    
    # Start a new MLflow run
    with mlflow.start_run() as run:
        
        # We train the model
        number_of_epochs = 3
        history = model.fit(train_dataset, validation_data=val_dataset, epochs=number_of_epochs)
        
        # Unable autologging
        mlflow.tensorflow.autolog()
        # Log other parameters    
        mlflow.log_param("model_name", model_name)
        mlflow.log_param("labels", labels)
        mlflow.log_param("start_date", start_date)
        mlflow.log_param("end_date", end_date)
        # Log the dataset as artifact
        mlflow.log_artifact("dataset_json.json")
        # For the other parameters and artifact we use the mlflow.tensorflow.autolog (but of course this could be customized if needed)'''

2024-12-26 17:48:00.525330: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/3
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 836ms/step - accuracy: 0.9683 - loss: 0.1100

5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.9667 - loss: 0.1211 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/3
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 753ms/step - accuracy: 1.0000 - loss: 0.0725 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/3
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 785ms/step - accuracy: 0.9778 - loss: 0.0451 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step


In [23]:
# We end the MLflow run
if experiment_name is not None:
    mlflow.end_run()